In [2]:
import pandas as pd
import numpy as np
import datetime as dt

open_time = dt.time(9,30)
close_time = dt.time(15,0)
moon_break_time = dt.time(11,30)
moon_open_time = dt.time(13,1)

# 读入数据
data_csv = pd.read_csv('C:\\Users\\longf.DESKTOP-7QSFE46\\GitHub\\A-Stock-Prediction-System-with-GAN-and-DRL\\dataset\\News_with_code-2019-07-28-to-2019-01-30.csv', 
                       nrows=3000)
x_data = data_csv.set_index(
                        pd.to_datetime(data_csv['datetime'])).drop(
                            columns=[x for x in data_csv.columns if x.startswith('Unnamed: ')]).sort_index(ascending=True)

data_csv = pd.read_csv('C:\\Users\\longf.DESKTOP-7QSFE46\\GitHub\\A-Stock-Prediction-System-with-GAN-and-DRL\\dataset\\MinutesStock-600690.SH-6 months from 2019-07-29 .csv',
                      usecols=["trade_time", "open", "high", "low", "close", "vol"])
y_data = data_csv.set_index(
                        pd.to_datetime(data_csv['trade_time'])).drop(
                            columns=[x for x in data_csv.columns if x.startswith('Unnamed: ')]).sort_index(ascending=True)
y_data = y_data.drop(columns=['trade_time'])

# 处理开市和休市索引
x_open_market_index = [x for x in x_data.index 
                                if (x.time() >= open_time and x.time() <= moon_break_time) 
                                or (x.time() >= moon_open_time and x.time() <= close_time)
                        ]
y_open_market_index = [x for x in y_data.index 
                                if (x.time() >= open_time and x.time() <= moon_break_time) 
                                or (x.time() >= moon_open_time and x.time() <= close_time)
                        ]
x_close_market_index = [x for x in x_data.index if x not in x_open_market_index ]
y_close_market_index = [x for x in y_data.index if x not in y_open_market_index ]
y_fluctuation = y_data - y_data.shift(1)

final_index =   [
                        x_data.loc[x_open_market_index], 
                        y_fluctuation.loc[y_open_market_index], 
                        x_data.loc[x_close_market_index], 
                        y_fluctuation.loc[y_close_market_index]
                        ]


In [3]:
[x_open, y_open, x_close, y_close] = final_index

# 区分开市和休市时间段，对应于x，y的交集
open_market_index_start = max(y_open.index.min(), x_open.index.min())
open_market_index_end = min(y_open.index.max(), x_open.index.max())
close_market_index_start = max(y_close.index.min(), x_close.index.min())
close_market_index_end = min(y_close.index.max(), x_close.index.max())

# 整个数据集的索引，便于建立x，y对应关系
market_start = min(open_market_index_start, close_market_index_start)
market_end = max(open_market_index_end, close_market_index_end)
market_range = pd.date_range(start=market_start, end=market_end, freq='Min')
market_range_daily = pd.date_range(start=market_start, end=market_end, freq="D")

# 定义数据窗口的参数
window_len = 60
open_market_step = 1
close_market_step = 6
open_market_y_len = 60
close_market_y_len = 1
        
# 日期的截断和加减
window_start = market_start.replace(second=0)-dt.timedelta(minutes=1)
window_end = window_start+dt.timedelta(minutes=61)
print(window_start, window_end)
print(type(window_start))
print(type(x_data.index[0]))

# 生成一个窗口的时段
window_index = pd.period_range(window_start, window_end, freq='Min')
print(window_index[:10])
print(x_data.index[:10])

# 布尔运算这里 为啥只出现后半部分的结果？？
# saoga 原因是 python中and的判断是根据表达式两端的True或False，比如前True后False，如果是and，就判断后边表达式并输出后面的结果
# 如果是or判断，如果第一个表达式为True，那么不再判断第二个表达式，直接输出第一个表达式
print([x_data.index > window_start])
print([x_data.index < window_end])
# 利用列表生成式先生成索引，在选择数据，因为列表生成式可以用布尔运算
idx = [x for x in x_data.index if x >= window_start and x <= window_end]
print(idx)
print(x_data.loc[idx])

y_idx = [x for x in y_data.index if x.time() in [open_time, close_time, moon_break_time, moon_open_time]]
print(y_idx)
y_open_close = y_data.loc[y_idx]
print(y_open_close)
y_open_close_delta = y_open_close - y_open_close.shift(1)
print(y_open_close_delta)

2019-07-21 10:00:00 2019-07-21 11:01:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
PeriodIndex(['2019-07-21 10:00', '2019-07-21 10:01', '2019-07-21 10:02',
             '2019-07-21 10:03', '2019-07-21 10:04', '2019-07-21 10:05',
             '2019-07-21 10:06', '2019-07-21 10:07', '2019-07-21 10:08',
             '2019-07-21 10:09'],
            dtype='period[T]', freq='T')
DatetimeIndex(['2019-07-20 00:10:23', '2019-07-20 00:11:34',
               '2019-07-20 00:12:14', '2019-07-20 00:51:06',
               '2019-07-20 00:54:45', '2019-07-20 01:02:49',
               '2019-07-20 01:11:55', '2019-07-20 01:21:13',
               '2019-07-20 01:31:10', '2019-07-20 01:49:37'],
              dtype='datetime64[ns]', name='datetime', freq=None)
[array([False, False, False, ...,  True,  True,  True])]
[array([ True,  True,  True, ..., False, False, False])]
[Timestamp('2019-07-21 10:01:10'), Timestamp('2019-07-21 10:13:21'), Timestamp

In [99]:
def _in_which_time(timeindex):
        open_time = dt.time(9,30)
        close_time = dt.time(15,0)
        moon_break_time = dt.time(11,30)
        moon_open_time = dt.time(13,1)

        if timeindex.time() >= open_time and timeindex.time() < moon_break_time:
            return 'morning_open'
        elif timeindex.time() >= moon_break_time and timeindex.time() < moon_open_time:
            return 'noon_break'
        elif timeindex.time() >= moon_open_time and timeindex.time() < close_time:
            return 'afternoon_open'
        elif timeindex.time() >= close_time or timeindex.time() < open_time:
            return 'night_break'
iloc = 1
print(market_range[iloc], _in_which_time(market_range[iloc]))
if _in_which_time(market_range[iloc]) == _in_which_time(market_range[iloc] + dt.timedelta(minutes=60)) == 'morning_open':
    print('morning_open')

2019-07-21 10:02:10 morning_open
morning


In [ ]:
a = (3,4,5)
print(a)